We initialize Python imports and opens a DuckDB connection that every later cell reuses

In [11]:
import warnings
from pathlib import Path
import duckdb
import numpy as np
import pandas as pd
import statsmodels.api as sm
import statsmodels.formula.api as smf

try:
    import statsmodels.genmod.generalized_linear_model as glm
    glm.SET_USE_BIC_LLF(True)
except Exception:
    pass

warnings.filterwarnings(
    "ignore",
    category=FutureWarning,
    message="The bic value is computed using the deviance formula.*",
)

CWD = Path().resolve()

REPO_ROOT = None
DB_FILE = None

for p in [CWD] + list(CWD.parents):
    cand = p / "db" / "nflpa.duckdb"
    if cand.exists():
        REPO_ROOT = p
        DB_FILE = cand
        break

if DB_FILE is None:
    for p in [CWD] + list(CWD.parents):
        cand = p / "nflpa.duckdb"
        if cand.exists():
            REPO_ROOT = p
            DB_FILE = cand
            break

if DB_FILE is None:
    raise RuntimeError("Could not find nflpa.duckdb, expected db/nflpa.duckdb under the repo root")

try:
    if "con" in globals():
        try:
            con.close()
        except Exception:
            pass
except Exception:
    pass

try:
    con = duckdb.connect(str(DB_FILE))
except Exception as e:
    msg = str(e)
    if "Could not set lock on file" in msg or "Conflicting lock is held" in msg:
        raise RuntimeError(
            "DuckDB is locked by another python process. Close other notebook kernels using nflpa.duckdb, or kill the PID shown in the error, then rerun."
        ) from e
    raise

SEASON_COL = "season"
WEEK_COL = "week"
TEAM_COL = "team"

MODEL_VIEW = "team_week_panel_nextweek_model"

exists_df = con.execute(f"""
SELECT
  COUNT(*) AS n
FROM information_schema.tables
WHERE table_schema = 'main'
  AND table_name = '{MODEL_VIEW}'
  AND table_type IN ('BASE TABLE', 'VIEW')
""").df()

if int(exists_df["n"].iloc[0]) == 0:
    raise RuntimeError(f"Missing {MODEL_VIEW}, run notebook 11 to create the model view")

print("connected db", str(DB_FILE))
print("model view", MODEL_VIEW)

connected db /Users/ramko/Desktop/2025-26-NFLPA-Data-Analytics-Case-Competition/db/nflpa.duckdb
model view team_week_panel_nextweek_model


Quick sanity check to confirm that 'has_next_week' is always 1 in the model view and that the view has unique season week team keys

In [12]:
con.execute(f"""
SELECT
  COUNT(*) AS rows_model,
  SUM(CASE WHEN has_next_week != 1 THEN 1 ELSE 0 END) AS bad_has_next_week,
  SUM(CASE WHEN Inj_Def_Next_w IS NULL THEN 1 ELSE 0 END) AS null_outcome_def
FROM {MODEL_VIEW}
""").df()

con.execute(f"""
SELECT
  COUNT(*) AS dup_rows
FROM (
  SELECT
    season,
    week,
    team AS team_key,
    COUNT(*) AS n
  FROM {MODEL_VIEW}
  GROUP BY 1,2,3
  HAVING COUNT(*) > 1
) d
""").df()

check_has_next = con.execute(f"""
SELECT
  COUNT(*) AS rows_model,
  SUM(CASE WHEN has_next_week != 1 THEN 1 ELSE 0 END) AS bad_has_next_week,
  SUM(CASE WHEN Inj_Def_Next_w IS NULL THEN 1 ELSE 0 END) AS null_outcome_def
FROM {MODEL_VIEW}
""").df()

check_dups = con.execute(f"""
SELECT
  COUNT(*) - COUNT(DISTINCT season || '-' || week || '-' || team) AS n_dup_keys
FROM {MODEL_VIEW}
""").df()

print(check_has_next.to_string(index=False))
print()
print(check_dups.to_string(index=False))

 rows_model  bad_has_next_week  null_outcome_def
       5950                0.0               0.0

 n_dup_keys
          0


Quick sanity check to confirm that all required columns exist. We normalize any naming differences so formulas are stable

In [3]:
cols = con.execute(f"DESCRIBE {MODEL_VIEW}").df()["column_name"].tolist()
cols_set = set(cols)

def pick_first_present(options):
    for o in options:
        if o in cols_set:
            return o
    return None

TEAM_COL = "team" if "team" in cols_set else ("team_key" if "team_key" in cols_set else None)
if TEAM_COL is None:
    raise RuntimeError("Missing team column, expected team or team_key in model view")

POINTS_FOR_COL = pick_first_present(["points_for_w", "points_for"])
POINTS_AGAINST_COL = pick_first_present(["points_against_w", "points_against"])
SCORE_DIFF_COL = pick_first_present(["score_diff_w", "score_diff"])
OFF_YPP_COL = pick_first_present(["off_yards_per_play_w", "Off_yards_per_play_w"])
CWI_COL = pick_first_present(["Cumulative_Workload_Index_w", "cumulative_workload_index_w"])

if POINTS_FOR_COL is None or POINTS_AGAINST_COL is None:
    raise RuntimeError("Missing points columns, expected points_for and points_against variants")

if SCORE_DIFF_COL is None:
    raise RuntimeError("Missing score diff column, expected score_diff_w")

if OFF_YPP_COL is None:
    raise RuntimeError("Missing offensive yards per play column, expected off_yards_per_play_w")

if CWI_COL is None:
    raise RuntimeError("Missing workload index column, expected Cumulative_Workload_Index_w")

OUTCOME_DEF = "Inj_Def_Next_w"
LAG_COLS = [
    "ST_Shock_NonScore_w_minus_1",
    "ST_Shock_NonScore_w_minus_2",
    "ST_Shock_NonScore_w_minus_3",
]

SHOCK_COL_MAIN = "ST_Shock_NonScore_Roll_w" if "ST_Shock_NonScore_Roll_w" in cols_set else "ST_Shock_NonScore_w"

required = [
    SEASON_COL, WEEK_COL, TEAM_COL,
    OUTCOME_DEF,
    "Inj_Def_Last_w",
    "blowout_flag_w",
    "short_week_flag_w",
    "bye_last_week_flag_w",
    "home_flag_w",
    "offensive_snaps_w",
    "defensive_snaps_w",
    SHOCK_COL_MAIN,
    "ST_Vol_NonScore_w",
    "Cum_Shocks_NonScore_w",
    OFF_YPP_COL,
    SCORE_DIFF_COL,
    POINTS_FOR_COL,
    POINTS_AGAINST_COL,
    CWI_COL,
] + LAG_COLS

missing = [c for c in required if c not in cols_set]
if missing:
    raise RuntimeError("Missing required Step 14 columns in model view, " + ", ".join(missing))

print("team column", TEAM_COL)
print("shock column main", SHOCK_COL_MAIN)
print("points_for column", POINTS_FOR_COL)
print("points_against column", POINTS_AGAINST_COL)
print("score_diff column", SCORE_DIFF_COL)
print("off_ypp column", OFF_YPP_COL)
print("workload index column", CWI_COL)

team column team
shock column main ST_Shock_NonScore_w
points_for column points_for
points_against column points_against
score_diff column score_diff_w
off_ypp column off_yards_per_play_w
workload index column Cumulative_Workload_Index_w


We build the modeling frame from the model view, then construct shock variables and interactions, and finally drop rows only where truly required inputs are missing

In [4]:
select_cols = [
    SEASON_COL,
    WEEK_COL,
    TEAM_COL,
    OUTCOME_DEF,
    "Inj_Def_Last_w",
    "blowout_flag_w",
    "short_week_flag_w",
    "bye_last_week_flag_w",
    "home_flag_w",
    "offensive_snaps_w",
    "defensive_snaps_w",
    SHOCK_COL_MAIN,
    "ST_Vol_NonScore_w",
    "Cum_Shocks_NonScore_w",
] + LAG_COLS + [
    POINTS_FOR_COL,
    POINTS_AGAINST_COL,
    SCORE_DIFF_COL,
    OFF_YPP_COL,
    CWI_COL,
]

df = con.execute(f"SELECT {', '.join(select_cols)} FROM {MODEL_VIEW}").df()

rename_map = {}
if POINTS_FOR_COL != "points_for":
    rename_map[POINTS_FOR_COL] = "points_for"
if POINTS_AGAINST_COL != "points_against":
    rename_map[POINTS_AGAINST_COL] = "points_against"
if SCORE_DIFF_COL != "score_diff_w":
    rename_map[SCORE_DIFF_COL] = "score_diff_w"
if OFF_YPP_COL != "off_yards_per_play_w":
    rename_map[OFF_YPP_COL] = "off_yards_per_play_w"
if CWI_COL != "Cumulative_Workload_Index_w":
    rename_map[CWI_COL] = "Cumulative_Workload_Index_w"

df = df.rename(columns=rename_map)

df[TEAM_COL] = df[TEAM_COL].astype(str)
df[SEASON_COL] = df[SEASON_COL].astype(int)
df[WEEK_COL] = df[WEEK_COL].astype(int)

df["season_week"] = (df[SEASON_COL] * 100 + df[WEEK_COL]).astype(int)

df["shock_nonscore"] = df[SHOCK_COL_MAIN].fillna(0).astype(int)
df["blowout_flag_w"] = df["blowout_flag_w"].fillna(0).astype(int)
df["shock_x_blowout"] = (df["shock_nonscore"] * df["blowout_flag_w"]).astype(int)

for c in LAG_COLS:
    df[c] = df[c].fillna(0).astype(int)

flag_cols = ["short_week_flag_w", "bye_last_week_flag_w", "home_flag_w"]
for c in flag_cols:
    df[c] = df[c].fillna(0).astype(int)

df["ST_Vol_NonScore_w"] = df["ST_Vol_NonScore_w"].fillna(0).astype(float)
df["Cum_Shocks_NonScore_w"] = df["Cum_Shocks_NonScore_w"].fillna(0).astype(float)

must_not_be_null = [
    OUTCOME_DEF,
    "Inj_Def_Last_w",
    "offensive_snaps_w",
    "defensive_snaps_w",
    "points_for",
    "points_against",
    "score_diff_w",
    "off_yards_per_play_w",
    "Cumulative_Workload_Index_w",
]

before = len(df)
df = df.dropna(subset=must_not_be_null).reset_index(drop=True)
after = len(df)

print("rows before dropna", before)
print("rows after dropna", after)
df.head(3)

rows before dropna 5950
rows after dropna 5950


,season,week,team,Inj_Def_Next_w,Inj_Def_Last_w,blowout_flag_w,short_week_flag_w,bye_last_week_flag_w,home_flag_w,offensive_snaps_w,...,ST_Shock_NonScore_w_minus_2,ST_Shock_NonScore_w_minus_3,points_for,points_against,score_diff_w,off_yards_per_play_w,Cumulative_Workload_Index_w,season_week,shock_nonscore,shock_x_blowout
0,2012,1,ATL,2.0,0.0,1,0,0,0,55.0,...,0,0,40,24,16,6.836364,-3.940011,201201,0,0
1,2012,2,ATL,3.0,2.0,0,0,0,1,65.0,...,0,0,27,21,6,4.412698,-3.638251,201202,0,0
2,2012,3,ATL,2.0,3.0,1,0,0,0,69.0,...,0,0,27,3,24,5.565217,-2.938346,201203,0,0


Quick sanity check to confirm that the modeling frame has no nulls in key predictors and that core numeric fields look finite and usable

In [5]:
check_cols = [
    OUTCOME_DEF,
    "shock_nonscore",
    "ST_Vol_NonScore_w",
    "Cum_Shocks_NonScore_w",
    "points_for",
    "points_against",
    "score_diff_w",
    "offensive_snaps_w",
    "defensive_snaps_w",
    "off_yards_per_play_w",
    "Inj_Def_Last_w",
    "Cumulative_Workload_Index_w",
]

null_counts = {c: int(df[c].isna().sum()) for c in check_cols}
nonfinite = {
    "ST_Vol_NonScore_w_nonfinite": int((~np.isfinite(df["ST_Vol_NonScore_w"].astype(float))).sum()),
    "Cum_Shocks_NonScore_w_nonfinite": int((~np.isfinite(df["Cum_Shocks_NonScore_w"].astype(float))).sum()),
    "off_yards_per_play_w_nonfinite": int((~np.isfinite(df["off_yards_per_play_w"].astype(float))).sum()),
    "CWI_nonfinite": int((~np.isfinite(df["Cumulative_Workload_Index_w"].astype(float))).sum()),
}

print("null counts", null_counts)
print("nonfinite counts", nonfinite)
print("n rows", len(df))
print("n teams", df[TEAM_COL].nunique())

null counts {'Inj_Def_Next_w': 0, 'shock_nonscore': 0, 'ST_Vol_NonScore_w': 0, 'Cum_Shocks_NonScore_w': 0, 'points_for': 0, 'points_against': 0, 'score_diff_w': 0, 'offensive_snaps_w': 0, 'defensive_snaps_w': 0, 'off_yards_per_play_w': 0, 'Inj_Def_Last_w': 0, 'Cumulative_Workload_Index_w': 0}
nonfinite counts {'ST_Vol_NonScore_w_nonfinite': 0, 'Cum_Shocks_NonScore_w_nonfinite': 0, 'off_yards_per_play_w_nonfinite': 0, 'CWI_nonfinite': 0}
n rows 5950
n teams 35


We compute dispersion diagnostics for the defensive next week injury count so the Poisson baseline and Negative Binomial contingency choice is explicitly grounded in the data

In [13]:
def outcome_dispersion_stats(y: pd.Series) -> dict:
    y = y.astype(float)
    mean = float(y.mean())
    var = float(y.var(ddof=1))
    share_zero = float((y == 0).mean())
    return {
        "mean": mean,
        "var": var,
        "var_over_mean": (var / mean) if mean > 0 else np.nan,
        "share_zero": share_zero,
        "max": float(y.max()),
    }

stats_def = outcome_dispersion_stats(df[OUTCOME_DEF])

print("defense outcome", OUTCOME_DEF)
for k in ["mean", "var", "var_over_mean", "share_zero", "max"]:
    print(k, stats_def[k])

material_overdispersion_def = (not np.isnan(stats_def["var_over_mean"])) and (stats_def["var_over_mean"] >= 1.5)
print("material overdispersion defense", bool(material_overdispersion_def))

defense outcome Inj_Def_Next_w
mean 2.083865546218487
var 2.380761656150105
var_over_mean 1.1424737361152615
share_zero 0.15529411764705883
max 10.0
material overdispersion defense False


We fit Model A using Poisson with team clustered standard errors and also fit Negative Binomial on the same formulas as a contingency and direct comparison

In [7]:
FE_TEAM = f"C({TEAM_COL})"
FE_TIME = "C(season_week)"
cluster_groups = df[TEAM_COL]

def fit_count_model_poisson_glm(formula: str, data: pd.DataFrame, groups: pd.Series):
    m = smf.glm(formula=formula, data=data, family=sm.families.Poisson())
    r = m.fit(cov_type="cluster", cov_kwds={"groups": groups})
    return r

def fit_count_model_negative_binomial_mle(formula: str, data: pd.DataFrame, groups: pd.Series, maxiter: int = 200):
    m = smf.negativebinomial(formula=formula, data=data)
    r = m.fit(disp=False, maxiter=maxiter, cov_type="cluster", cov_kwds={"groups": groups})
    return r

def _safe_float(x):
    try:
        return float(x)
    except Exception:
        return np.nan

def print_nb_compare(pois_res, nb_res, name: str):
    if nb_res is None:
        print(name, "negative binomial fit failed")
        return
    aic_pois = _safe_float(getattr(pois_res, "aic", np.nan))
    aic_nb = _safe_float(getattr(nb_res, "aic", np.nan))
    alpha = np.nan
    try:
        if "alpha" in nb_res.params.index:
            alpha = _safe_float(nb_res.params["alpha"])
    except Exception:
        pass
    print(name, "aic_poisson", aic_pois, "aic_nb", aic_nb, "alpha", alpha)

exposure_terms = [
    "shock_nonscore",
    "shock_x_blowout",
    "ST_Vol_NonScore_w",
    "Cum_Shocks_NonScore_w",
    "ST_Shock_NonScore_w_minus_1",
    "ST_Shock_NonScore_w_minus_2",
    "ST_Shock_NonScore_w_minus_3",
]

control_terms_base = [
    "offensive_snaps_w",
    "defensive_snaps_w",
    "blowout_flag_w",
    "short_week_flag_w",
    "bye_last_week_flag_w",
    "home_flag_w",
    "off_yards_per_play_w",
    "Inj_Def_Last_w",
    "Cumulative_Workload_Index_w",
]

script_specs = [
    ("points_for_against", ["points_for", "points_against"]),
    ("points_for_diff", ["points_for", "score_diff_w"]),
    ("points_against_diff", ["points_against", "score_diff_w"]),
]

def build_formula(outcome: str, script_terms: list) -> str:
    rhs = exposure_terms + control_terms_base + script_terms + [FE_TEAM, FE_TIME]
    return outcome + " ~ " + " + ".join(rhs)

fits = []

for tag, script_terms in script_specs:
    f = build_formula(OUTCOME_DEF, script_terms)
    try:
        pois = fit_count_model_poisson_glm(f, df, cluster_groups)
    except Exception as e:
        print("poisson failed", tag, str(e))
        continue

    nb = None
    try:
        nb = fit_count_model_negative_binomial_mle(f, df, cluster_groups)
    except Exception as e:
        print("negative binomial failed", tag, str(e))

    fits.append((tag, f, pois, nb))
    print("fit ok", tag)
    print_nb_compare(pois, nb, "nb compare " + tag)
    print()

if len(fits) == 0:
    raise RuntimeError("No Model A specifications fit successfully")

preferred_order = ["points_for_against", "points_for_diff", "points_against_diff"]
fits_sorted = sorted(fits, key=lambda x: preferred_order.index(x[0]) if x[0] in preferred_order else 999)
spec_tag_def, formula_def_used, pois_def, nb_def = fits_sorted[0]

print("selected Model A spec", spec_tag_def)
print(formula_def_used)

fit ok points_for_against
nb compare points_for_against aic_poisson 20850.174345191903 aic_nb 20851.513907952096 alpha 0.007044369821692421

fit ok points_for_diff
nb compare points_for_diff aic_poisson 20850.174345191903 aic_nb 20851.513934615534 alpha 0.007040657560628567

fit ok points_against_diff
nb compare points_against_diff aic_poisson 20850.174345191903 aic_nb 20851.51390948031 alpha 0.007043686356937115

selected Model A spec points_for_against
Inj_Def_Next_w ~ shock_nonscore + shock_x_blowout + ST_Vol_NonScore_w + Cum_Shocks_NonScore_w + ST_Shock_NonScore_w_minus_1 + ST_Shock_NonScore_w_minus_2 + ST_Shock_NonScore_w_minus_3 + offensive_snaps_w + defensive_snaps_w + blowout_flag_w + short_week_flag_w + bye_last_week_flag_w + home_flag_w + off_yards_per_play_w + Inj_Def_Last_w + Cumulative_Workload_Index_w + points_for + points_against + C(team) + C(season_week)


We print and interpret the key exposure coefficients as IRR with clustered confidence intervals for the selected Model A specification

In [8]:
key_terms = ["shock_nonscore", "shock_x_blowout", "ST_Vol_NonScore_w", "Cum_Shocks_NonScore_w"]

def print_key_terms(res, name: str):
    print(name, "nobs", int(res.nobs))
    for t in key_terms:
        if t in res.params.index:
            beta = float(res.params[t])
            se = float(res.bse[t])
            p = float(res.pvalues[t])
            irr = float(np.exp(beta))
            ci_lo = float(np.exp(beta - 1.96 * se))
            ci_hi = float(np.exp(beta + 1.96 * se))
            print(t, "beta", beta, "se", se, "irr", irr, "ci", (ci_lo, ci_hi), "p", p)

print_key_terms(pois_def, "Model A poisson selected")
print()
if nb_def is not None:
    print_key_terms(nb_def, "Model A negative binomial selected")

Model A poisson selected nobs 5950
shock_nonscore beta 0.026332369954740246 se 0.03735736398489226 irr 1.0266821300641484 ci (0.954194200814147, 1.104676799852365) p 0.480886396415659
shock_x_blowout beta 0.07834013547131219 se 0.06672495526291138 irr 1.0814904493809236 ci (0.9489105168673143, 1.2325941922990615) p 0.24036478427274077
ST_Vol_NonScore_w beta -0.00793820007087621 se 0.01295282917533205 irr 0.9920932242335248 ci (0.9672234398218448, 1.0176024743066214) p 0.5399725292329625
Cum_Shocks_NonScore_w beta -0.01979545573109645 se 0.016441364781304188 irr 0.9803991878341166 ci (0.949309376654469, 1.0125071880077385) p 0.22858834355065982

Model A negative binomial selected nobs 5950
shock_nonscore beta 0.02622262001156779 se 0.037361524562687876 irr 1.0265694579417102 ci (0.9540817034885019, 1.104564575680952) p 0.482765468360963
shock_x_blowout beta 0.07853422268540877 se 0.06680378998445137 irr 1.0817003732204238 ci (0.9489480671000355, 1.2330239535667422) p 0.2397566276569869


We export Model A results to DuckDB and CSV in a tidy format, including the both Poisson and Negative Binomial rows when available

In [9]:
def tidy_res(res, model_name: str, outcome_name: str, spec_tag: str, irr_terms: list) -> pd.DataFrame:
    params = res.params.copy()
    bse = res.bse.copy()
    pvals = res.pvalues.copy()

    out = pd.DataFrame({
        "model": model_name,
        "spec_tag": spec_tag,
        "outcome": outcome_name,
        "term": params.index.astype(str),
        "beta": params.values.astype(float),
        "se_cluster": bse.values.astype(float),
        "pvalue": pvals.values.astype(float),
    })

    irr_terms_set = set(irr_terms)
    out["is_irr_term"] = out["term"].apply(lambda x: 1 if x in irr_terms_set else 0)

    out["irr"] = np.nan
    out["ci_low_irr"] = np.nan
    out["ci_high_irr"] = np.nan

    mask = out["is_irr_term"] == 1
    out.loc[mask, "irr"] = np.exp(out.loc[mask, "beta"])
    out.loc[mask, "ci_low_irr"] = np.exp(out.loc[mask, "beta"] - 1.96 * out.loc[mask, "se_cluster"])
    out.loc[mask, "ci_high_irr"] = np.exp(out.loc[mask, "beta"] + 1.96 * out.loc[mask, "se_cluster"])

    out["nobs"] = int(res.nobs)
    out["aic"] = float(getattr(res, "aic", np.nan))
    out["bic"] = float(getattr(res, "bic", np.nan))
    out["llf"] = float(getattr(res, "llf", np.nan))
    return out

results = []
results.append(tidy_res(pois_def, "poisson_modelA", OUTCOME_DEF, spec_tag_def, key_terms))
if nb_def is not None:
    results.append(tidy_res(nb_def, "negative_binomial_modelA", OUTCOME_DEF, spec_tag_def, key_terms))

results_df = pd.concat(results, ignore_index=True)
key_keep = set(key_terms)
results_df["is_key_term"] = results_df["term"].apply(lambda x: 1 if x in key_keep else 0)

con.register("step14_modelA_tmp", results_df)
con.execute("CREATE OR REPLACE TABLE step14_modelA_results AS SELECT * FROM step14_modelA_tmp")
con.unregister("step14_modelA_tmp")

out_dir = Path("../outputs")
out_dir.mkdir(parents=True, exist_ok=True)
csv_path = out_dir / "step14_modelA_results.csv"
results_df.to_csv(csv_path, index=False)

print("wrote duckdb table step14_modelA_results")
print("wrote csv", csv_path.resolve())

results_df.query("is_key_term == 1").sort_values(["model", "term"]).head(20)

wrote duckdb table step14_modelA_results
wrote csv /Users/ramko/Desktop/2025-26-NFLPA-Data-Analytics-Case-Competition/outputs/step14_modelA_results.csv


,model,spec_tag,outcome,term,beta,se_cluster,pvalue,is_irr_term,irr,ci_low_irr,ci_high_irr,nobs,aic,bic,llf,is_key_term
513,negative_binomial_modelA,points_for_against,Inj_Def_Next_w,Cum_Shocks_NonScore_w,-0.019758,0.016462,0.230054,1,0.980436,0.949307,1.012586,5950,20851.513908,22624.667730,-10160.756954,1
512,negative_binomial_modelA,points_for_against,Inj_Def_Next_w,ST_Vol_NonScore_w,-0.007919,0.012989,0.542061,1,0.992112,0.967174,1.017693,5950,20851.513908,22624.667730,-10160.756954,1
510,negative_binomial_modelA,points_for_against,Inj_Def_Next_w,shock_nonscore,0.026223,0.037362,0.482765,1,1.026569,0.954082,1.104565,5950,20851.513908,22624.667730,-10160.756954,1
511,negative_binomial_modelA,points_for_against,Inj_Def_Next_w,shock_x_blowout,0.078534,0.066804,0.239757,1,1.081700,0.948948,1.233024,5950,20851.513908,22624.667730,-10160.756954,1
249,poisson_modelA,points_for_against,Inj_Def_Next_w,Cum_Shocks_NonScore_w,-0.019795,0.016441,0.228588,1,0.980399,0.949309,1.012507,5950,20850.174345,22616.637021,-10161.087173,1
248,poisson_modelA,points_for_against,Inj_Def_Next_w,ST_Vol_NonScore_w,-0.007938,0.012953,0.539973,1,0.992093,0.967223,1.017602,5950,20850.174345,22616.637021,-10161.087173,1
246,poisson_modelA,points_for_against,Inj_Def_Next_w,shock_nonscore,0.026332,0.037357,0.480886,1,1.026682,0.954194,1.104677,5950,20850.174345,22616.637021,-10161.087173,1
247,poisson_modelA,points_for_against,Inj_Def_Next_w,shock_x_blowout,0.078340,0.066725,0.240365,1,1.081490,0.948911,1.232594,5950,20850.174345,22616.637021,-10161.087173,1
